In [ ]:
#@title import libraries
import torch
import numpy as np
import meent # meent version 10.0.0
import os
from tqdm import tqdm
import time
from IPython.display import clear_output, display
start = time.time()

P = 1000
t = 1500
h = 500
N_L = 4
N_C = 32
pixel_min_width = P/N_C
pixel_min_height = t/N_L


In [ ]:
n_air = 1.0

backend = 2  # torch
device = 0
pol = 1 # 0: TE, 1: TM


theta = 0 * torch.pi / 180  # angle of incidence
phi = 0 * torch.pi / 180  # angle of rotation
period = torch.tensor([P])  # length of the unit cell. Here it's 1D.

fourier_order = [80]
type_complex = torch.complex128

# Resolution in x and z direction
res_x = N_C*2
res_z = 15

n1 = 1.5
n2 = 2.0

n_I = 1.0  # n_incidence
n_II = n2  # n_transmission,


thickness = torch.tensor([500.] + [pixel_min_height for layer in range(N_L)] + [50. for i in range(10)] + [100000.])  # thickness of each layer, from top to bottom.

# ucell formation for meent
def return_device(patterning):
    Layer_air =  n_air * torch.ones(1,1,N_C)
    Layer_patterning = torch.from_numpy(np.expand_dims(patterning, axis=1))
    Layer_detector =  n2 * torch.ones(12,1,N_C)

    ucell = torch.cat((Layer_air,Layer_patterning,Layer_detector))
    return ucell

# Optical response calculation
def RGB(field_cell_meent_1d):

    R_index_i = 0
    R_index_f = 16
    
    G_index_i = 16
    G_index_f = 32

    B_index_i = 32
    B_index_f = 48

    I_index_i = 48
    I_index_f = 64
    
    B = torch.sum(torch.abs(field_cell_meent_1d[B_index_i:B_index_f])**2)
    G1 = torch.sum(torch.abs(field_cell_meent_1d[G_index_i:G_index_f])**2)
    R = torch.sum(torch.abs(field_cell_meent_1d[R_index_i:R_index_f])**2)
    G2 = torch.sum(torch.abs(field_cell_meent_1d[I_index_i:I_index_f])**2)

    G = G1 + G2

    return R, G, B


# def data_augmentation(patterning, field_cell_meent_color):

#     patterning_aug_list, field_cell_aug_list = [], []
#     patterning_aug_list.append(patterning)
#     field_cell_aug_list.append(field_cell_meent_color)

#     for i in range(N_C-1):
#         patterning = np.roll(patterning, -1, axis = 1)
#         patterning_aug_list.append(patterning)  
#         # print(patterning)
#         field_cell_meent_color = np.roll(field_cell_meent_color, -1, axis = 1)
#         # print(field_cell_meent_color)
#         field_cell_aug_list.append(field_cell_meent_color)

#     return patterning_aug_list, field_cell_aug_list


wav_len_step = 10
wav_len_list = [400 + i * wav_len_step for i in range((300) // wav_len_step + 1)]

result_folder_path = r'D:\Sunghyun Nam\SANZABOO\data_Sz'
os.makedirs(result_folder_path, exist_ok=True)
input_patterning_folder = os.path.join(result_folder_path, 'patternings')
output_result_folder = os.path.join(result_folder_path, 'true_results')
os.makedirs(input_patterning_folder, exist_ok=True)
os.makedirs(output_result_folder, exist_ok=True)

In [ ]:
num_samples = 10000
n_alreadymade = 0

# Sample generation

for epoch in tqdm(range(num_samples), dynamic_ncols=True) :
    
    random_array = np.ones((N_L, N_C)) *n1 + (n2-n1)* np.random.randint(2, size=(N_L, N_C))
    ucells = []
    
    field_cell_meent_Sz_color = []

    for color in range(len(wav_len_list)):
        wav_len = wav_len_list[color]

        ucell = return_device(random_array)
        ucells.append(ucell.tolist())
        mee = meent.call_mee(backend=backend, pol=pol, n_top=n_I, n_bot=n_II, theta=theta, phi=phi, fto=fourier_order, wavelength=wav_len, period=period, thickness=thickness, type_complex=type_complex, device=device, fourier_type=0) # After update

        mee.ucell = ucell

        de_ri, de_ti = mee.conv_solve()

        field_cell_meent = mee.calculate_field(res_x=res_x,res_z=res_z)
        observation = res_z * (1+ N_L + 10)
        field_cell_meent_Sz = 0.5 * torch.real(field_cell_meent[:observation,0,:,0] * torch.conj(field_cell_meent[:observation,0,:,1])) # Sz calculation

        field_cell_meent_Sz_color.append(field_cell_meent_Sz.numpy())
        
    field_cell_meent_Sz_color = np.array(field_cell_meent_Sz_color)
    
    name = str(epoch+n_alreadymade).zfill(8)
    patterning_path = os.path.join(input_patterning_folder, 'patterning_'+name+'.npy')
    result_path = os.path.join(output_result_folder, 'result_Sz_'+name+'.npy')
    np.save(patterning_path, random_array)
    np.save(result_path, field_cell_meent_Sz_color)

    clear_output()
    tqdm.write(f"Simulation {epoch + 1}/{num_samples} completed", end="")
    

end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start
print(elapsed_time)
